In [5]:
import pandas as pd
import numpy as np
import itertools

In [36]:
df = pd.read_csv('data/basic_data.csv', index_col=0)
df

,AAPL,MSFT,AAL,WMT
Date,,,,
2019-01-31,40.283466,100.540642,35.200802,90.260887
2019-02-28,42.087215,108.318222,35.158627,93.237244
2019-03-29,46.170757,114.032387,31.339823,92.356361
2019-04-30,48.776455,126.272949,33.727806,97.384697
2019-05-31,42.717785,120.024864,26.947300,96.569237
2019-06-28,48.293270,130.001068,32.271446,105.184685
2019-07-31,51.982613,132.242813,30.193249,105.079971
2019-08-30,51.126923,134.231308,26.128435,109.309113
2019-09-30,54.857212,135.370499,26.783880,113.537590


In [103]:
stocks = ['AAPL', 'MSFT', 'AAL', 'WMT']
price = df.iloc[-1]
budget = 1000
alpha = 0.005
t_cost = 0.01
init_budget = budget
monthly_returns = df[list(stocks)].pct_change().iloc[1:]
avg_monthly_returns = monthly_returns.mean(axis=0)
covariance_matrix = monthly_returns.cov()

print(f'price: {price}')

max_num_shares = (budget / df.iloc[-1]).astype(int)

print(f'max_num_shares: {max_num_shares}')

best_risk = np.inf
best_returns = np.inf
best_objective = np.inf
best_portfolio = []
best_c0 = None
best_c1 = None
best_lhs = 0
best_spending = np.inf
best_variance = np.inf
for i0 in zip(100*[stocks[0]], range(0, 100)):
    for i1 in zip(100*[stocks[1]], range(0, 100)):
        for i2 in zip(100*[stocks[2]], range(0, 100)):
            for i3 in zip(100*[stocks[3]], range(0, 100)):
                returns = 0
                risk = 0
                if i0[1] <= max_num_shares[0] and i1[1] <= max_num_shares[1] and i2[1] <= max_num_shares[2] and i3[1] <= max_num_shares[3]:
                    coeff = covariance_matrix[i0[0]][i1[0]] * price[i0[0]] * price[i1[0]]
                    risk += coeff * i0[1] * i1[1]
                    coeff = covariance_matrix[i0[0]][i2[0]] * price[i0[0]] * price[i2[0]]
                    risk += coeff * i0[1] * i2[1]
                    coeff = covariance_matrix[i0[0]][i3[0]] * price[i0[0]] * price[i3[0]]
                    risk += coeff * i0[1] * i3[1]
                    coeff = covariance_matrix[i1[0]][i2[0]] * price[i1[0]] * price[i2[0]]
                    risk += coeff * i1[1] * i2[1]
                    coeff = covariance_matrix[i1[0]][i3[0]] * price[i1[0]] * price[i3[0]]
                    risk += coeff * i1[1] * i3[1]
                    coeff = covariance_matrix[i2[0]][i3[0]] * price[i2[0]] * price[i3[0]]
                    risk += coeff * i2[1] * i3[1]
                    if risk < best_risk: best_risk = risk
                    returns = (price[i0[0]] * avg_monthly_returns[i0[0]] * i0[1] +
                               price[i1[0]] * avg_monthly_returns[i1[0]] * i1[1] +
                               price[i2[0]] * avg_monthly_returns[i2[0]] * i2[1] +
                               price[i3[0]] * avg_monthly_returns[i3[0]] * i3[1])
                    if returns < best_returns: best_returns = returns
                    objective = alpha * risk - returns

                    no_cost_test = (price[i0[0]] * i0[1] +
                                   price[i1[0]] * i1[1] +
                                   price[i2[0]] * i2[1] +
                                   price[i3[0]] * i3[1])

                    lhs = (2 * t_cost  + (1 - t_cost)) * (price[i0[0]] * i0[1] + price[i1[0]] * i1[1] + price[i2[0]] * i2[1] + price[i3[0]] * i3[1])

                    c0 = lhs <= budget
                    c1 = lhs >= budget - 0.003 * budget

                    spending = (price[i0[0]] * max(0, i0[1]) +
                                price[i1[0]] * max(0, i1[1]) +
                                price[i2[0]] * max(0, i2[1]) +
                                price[i3[0]] * max(0, i3[1]))

                    variance = 0
                    for i in [i0, i1, i2, i3]:
                        for j in [i0, i1, i2, i3]:
                            variance += price[i[0]] * i[1] * price[j[0]] * j[1] * covariance_matrix[i[0]][j[0]]

                    if objective < best_objective and c0 and c1:
                        best_objective = objective
                        best_portfolio = [i0, i1, i2, i3]
                        best_lhs = lhs
                        best_c0 = c0
                        best_c1 = c1
                        best_spending = spending
                        best_variance = variance


print(f'objective: {best_objective}')
print(f'combination: {best_portfolio}')
print(f'lhs: {best_lhs}')
print(f'c0: {best_c0}')
print(f'c1: {best_c1}')
print(f'best variance: {best_variance}')
print(f'best spending: {best_spending}')

price: AAPL    157.418701
MSFT    276.875122
AAL      18.770000
WMT     152.435913
Name: 2022-04-29, dtype: float64
max_num_shares: AAPL     6
MSFT     3
AAL     53
WMT      6
Name: 2022-04-29, dtype: int64
objective: -26.97040025089213
combination: [('AAPL', 1), ('MSFT', 3), ('AAL', 0), ('WMT', 0)]
lhs: 997.9245080566407
c0: True
c1: True
best variance: 3780.486736198098
best spending: 988.0440673828125


In [ ]:
stocks = ['AAPL', 'MSFT', 'AAL', 'WMT']
i0 = ['AAPL', 2]
i1 = ['MSFT', 0]
i2 = ['AAL', 4]
i2 = ['WMT', 4]
price = df.iloc[-1]
budget = 1000
alpha = 0.005
t_cost = 0.01
init_budget = budget
monthly_returns = df[list(stocks)].pct_change().iloc[1:]
avg_monthly_returns = monthly_returns.mean(axis=0)
covariance_matrix = monthly_returns.cov()

print(f'price: {price}')

max_num_shares = (budget / df.iloc[-1]).astype(int)

print(f'max_num_shares: {max_num_shares}')

best_risk = np.inf
best_returns = np.inf
best_objective = np.inf
best_portfolio = []
keep_t0 = 0
keep_t1 = 0
best_c0 = None
best_c1 = None
best_lhs = 0
for i0 in zip(100 * [stocks[0]], range(0, 100)):
    for i1 in zip(100 * [stocks[1]], range(0, 100)):
        for i2 in zip(100 * [stocks[2]], range(0, 100)):
            for i3 in zip(100 * [stocks[3]], range(0, 100)):
                returns = 0
                risk = 0
                if i0[1] <= max_num_shares[0] and i1[1] <= max_num_shares[1] and i2[1] <= max_num_shares[2] and i3[1] <=
                        max_num_shares[3]:
                    coeff = covariance_matrix[i0[0]][i1[0]] * price[i0[0]] * price[i1[0]]
                    risk += coeff * i0[1] * i1[1]
                    coeff = covariance_matrix[i0[0]][i2[0]] * price[i0[0]] * price[i2[0]]
                    risk += coeff * i0[1] * i2[1]
                    coeff = covariance_matrix[i0[0]][i3[0]] * price[i0[0]] * price[i3[0]]
                    risk += coeff * i0[1] * i3[1]
                    coeff = covariance_matrix[i1[0]][i2[0]] * price[i1[0]] * price[i2[0]]
                    risk += coeff * i1[1] * i2[1]
                    coeff = covariance_matrix[i1[0]][i3[0]] * price[i1[0]] * price[i3[0]]
                    risk += coeff * i1[1] * i3[1]
                    coeff = covariance_matrix[i2[0]][i3[0]] * price[i2[0]] * price[i3[0]]
                    risk += coeff * i2[1] * i3[1]
                    if risk < best_risk: best_risk = risk
                    returns = (price[i0[0]] * avg_monthly_returns[i0[0]] * i0[1] +
                               price[i1[0]] * avg_monthly_returns[i1[0]] * i1[1] +
                               price[i2[0]] * avg_monthly_returns[i2[0]] * i2[1] +
                               price[i3[0]] * avg_monthly_returns[i3[0]] * i3[1])
                    if returns < best_returns: best_returns = returns
                    objective = alpha * risk - returns

                    no_cost_test = (price[i0[0]] * i0[1] +
                                    price[i1[0]] * i1[1] +
                                    price[i2[0]] * i2[1] +
                                    price[i3[0]] * i3[1])

                    #                 cost_test = 2 * t_cost * price[i0[0]] * x[s]
                    # print(2 * t_cost * price[i1[0]])
                    # print(2 * t_cost * price[i2[0]])
                    lhs = (2 * t_cost + (1 - t_cost)) * (
                                price[i0[0]] * i0[1] + price[i1[0]] * i1[1] + price[i2[0]] * i2[1] + price[i3[0]] * i3[
                            1])

                    c0 = lhs <= budget
                    c1 = lhs >= budget - 0.003 * budget
                    if objective < best_objective and c0 and c1:
                        best_objective = objective
                        best_portfolio = [i0, i1, i2, i3]
                        keep_t0 = t0
                        best_c0 = c0
                        best_c1 = c1

print(f'objective: {best_objective}')
print(f'combination: {best_portfolio}')
print(f't0: {keep_t0}')
print(f'c0: {best_c0}')
print(f'c1: {best_c1}')

In [65]:
stocks = ['AAPL', 'MSFT', 'AAL', 'WMT']
price = df.iloc[-1]
budget = 1000
alpha = 0.005
t_cost = 0.00
init_budget = budget
monthly_returns = df[list(stocks)].pct_change().iloc[1:]
avg_monthly_returns = monthly_returns.mean(axis=0)
covariance_matrix = monthly_returns.cov()

print(f'price: {price}')

max_num_shares = (budget / df.iloc[-1]).astype(int)

print(f'max_num_shares: {max_num_shares}')

best_risk = np.inf
best_returns = np.inf
best_objective = np.inf
best_portfolio = []
keep_t0 = 0
keep_t1 = 0
best_c0 = None
best_c1 = None
best_lhs = 0
for i0 in zip(100*[stocks[0]], range(0, 100)):
    for i1 in zip(100*[stocks[1]], range(0, 100)):
        for i2 in zip(100*[stocks[2]], range(0, 100)):
            for i3 in zip(100*[stocks[3]], range(0, 100)):
                returns = 0
                risk = 0
                if i0[1] <= max_num_shares[0] and i1[1] <= max_num_shares[1] and i2[1] <= max_num_shares[2] and i3[1] <= max_num_shares[3]:
                    coeff = covariance_matrix[i0[0]][i1[0]] * price[i0[0]] * price[i1[0]]
                    risk += coeff * i0[1] * i1[1]
                    coeff = covariance_matrix[i0[0]][i2[0]] * price[i0[0]] * price[i2[0]]
                    risk += coeff * i0[1] * i2[1]
                    coeff = covariance_matrix[i0[0]][i3[0]] * price[i0[0]] * price[i3[0]]
                    risk += coeff * i0[1] * i3[1]
                    coeff = covariance_matrix[i1[0]][i2[0]] * price[i1[0]] * price[i2[0]]
                    risk += coeff * i1[1] * i2[1]
                    coeff = covariance_matrix[i1[0]][i3[0]] * price[i1[0]] * price[i3[0]]
                    risk += coeff * i1[1] * i3[1]
                    coeff = covariance_matrix[i2[0]][i3[0]] * price[i2[0]] * price[i3[0]]
                    risk += coeff * i2[1] * i3[1]
                    if risk < best_risk: best_risk = risk
                    returns = (price[i0[0]] * avg_monthly_returns[i0[0]] * i0[1] +
                               price[i1[0]] * avg_monthly_returns[i1[0]] * i1[1] +
                               price[i2[0]] * avg_monthly_returns[i2[0]] * i2[1] +
                               price[i3[0]] * avg_monthly_returns[i3[0]] * i3[1])
                    if returns < best_returns: best_returns = returns
                    objective = alpha * risk - returns
                    lhs = 0
                    for b0 in [0,1]:
                        for b1 in [0,1]:
                            for b2 in [0,1]:
                                for b3 in [0,1]:
                                    t0 = (t_cost * price[i0[0]] * i0[1] * b0 +
                                          t_cost * price[i1[0]] * i1[1] * b1 +
                                          t_cost * price[i2[0]] * i2[1] * b2 +
                                          t_cost * price[i3[0]] * i3[1] * b3)
                                    t1 = (price[i0[0]] * (1 - t_cost) * i0[1] +
                                          price[i1[0]] * (1 - t_cost) * i1[1] +
                                          price[i2[0]] * (1 - t_cost) * i2[1] +
                                          price[i3[0]] * (1 - t_cost) * i3[1])
                                    lhs += 2 * t0 + t1
                    c0 = lhs <= budget
                    c1 = lhs >= budget - 0.003 * init_budget
                    if objective < best_objective:
                        best_objective = objective
                        best_portfolio = [i0, i1, i2, i3]
                        keep_t0 = t0
                        keep_t1 = t1
                        best_c0 = c0
                        best_c1 = c1
                        best_lhs = lhs

print(f'objective: {best_objective}')
print(f'combination: {best_portfolio}')
print(f't0: {keep_t0}')
print(f't1: {keep_t1}')
print(f'c0: {best_c0}')
print(f'c1: {best_c1}')
print(f'lhs: {best_lhs}')


price: AAPL    157.418701
MSFT    276.875122
AAL      18.770000
WMT     152.435913
Name: 2022-04-29, dtype: float64
max_num_shares: AAPL     6
MSFT     3
AAL     53
WMT      6
Name: 2022-04-29, dtype: int64
objective: -47.561564853020535
combination: [('AAPL', 6), ('MSFT', 3), ('AAL', 0), ('WMT', 6)]
t0: 26.897530517578126
t1: 2662.8555212402343
c0: False
c1: True
lhs: 43036.04882812499


In [78]:
i0 = ['AAPL', 2]
i1 = ['MSFT', 0]
i2 = ['AAL', 4]
i3 = ['WMT', 4]
returns = 0
risk = 0
if i0[1] <= max_num_shares[0] and i1[1] <= max_num_shares[1] and i2[1] <= max_num_shares[2] and i3[1] <= max_num_shares[3]:
    coeff = covariance_matrix[i0[0]][i1[0]] * price[i0[0]] * price[i1[0]]
    risk += coeff * i0[1] * i1[1]
    coeff = covariance_matrix[i0[0]][i2[0]] * price[i0[0]] * price[i2[0]]
    risk += coeff * i0[1] * i2[1]
    coeff = covariance_matrix[i0[0]][i3[0]] * price[i0[0]] * price[i3[0]]
    risk += coeff * i0[1] * i3[1]
    coeff = covariance_matrix[i1[0]][i2[0]] * price[i1[0]] * price[i2[0]]
    risk += coeff * i1[1] * i2[1]
    coeff = covariance_matrix[i1[0]][i3[0]] * price[i1[0]] * price[i3[0]]
    risk += coeff * i1[1] * i3[1]
    coeff = covariance_matrix[i2[0]][i3[0]] * price[i2[0]] * price[i3[0]]
    risk += coeff * i2[1] * i3[1]

    returns = (price[i0[0]] * avg_monthly_returns[i0[0]] * i0[1] +
                               price[i1[0]] * avg_monthly_returns[i1[0]] * i1[1] +
                               price[i2[0]] * avg_monthly_returns[i2[0]] * i2[1] +
                               price[i3[0]] * avg_monthly_returns[i3[0]] * i3[1])
    objective = alpha * risk - returns
    t0 = (price[i0[0]] * i0[1] +
          price[i1[0]] * i1[1] +
          price[i2[0]] * i2[1] +
          price[i3[0]] * i3[1])

    c0 = t0 <= budget
    c1 = t0 >= 0.997 * budget

    print(f'returns: {returns}')
    print(f'risk: {risk}')
    print(f'c0: {c0}')
    print(f'c1: {c1}')
              # lhs = 0
    # t0 = 0
    # t1 = (price[i0[0]] * (1 - t_cost) * i0[1] +
    #                            price[i1[0]] * (1 - t_cost) * i1[1] +
    #                            price[i2[0]] * (1 - t_cost) * i2[1] +
    #                            price[i3[0]] * (1 - t_cost) * i3[1])
    # # for b0 in [0,1]:
    # #     for b1 in [0,1]:
    # #         for b2 in [0,1]:
    # #             for b3 in [0,1]:
    # t0 = (t_cost * price[i0[0]] * i0[1] +
    #         t_cost * price[i1[0]] * i1[1] +
    #         t_cost * price[i2[0]] * i2[1] +
    #         t_cost * price[i3[0]] * i3[1])

    # lhs = 2 * t0 + t1
    # c0 = lhs <= budget
    # c1 = lhs >= budget - 0.003 * init_budget
    # print(b0, b1, b2, b3, lhs, c0, c1)
    # if c0 and c1: print(lhs)
#
# print(f'returns: {returns}')
# print(f'risk: {risk}')
# print(f'lhs: {lhs}')
# print(f'c0: {c0}')
# print(f'c1: {c1}')

returns: 20.755507334490737
risk: 408.65448996063867
c0: True
c1: True


In [91]:
avg_monthly_returns

AAPL    0.039129
MSFT    0.028096
AAL    -0.006494
WMT     0.014636
dtype: float64

In [99]:
155 + 274 + 150

579

In [100]:
i0 = ['AAPL', 1]
i1 = ['MSFT', 3]
i2 = ['WMT', 0]
returns = 0
risk = 0
t_cost = 0.01
if i0[1] <= max_num_shares[0] and i1[1] <= max_num_shares[1] and i2[1] <= max_num_shares[2]:
    coeff = covariance_matrix[i0[0]][i1[0]] * price[i0[0]] * price[i1[0]]
    risk += coeff * i0[1] * i1[1]
    coeff = covariance_matrix[i0[0]][i2[0]] * price[i0[0]] * price[i2[0]]
    risk += coeff * i0[1] * i2[1]
    coeff = covariance_matrix[i1[0]][i2[0]] * price[i1[0]] * price[i2[0]]
    risk += coeff * i1[1] * i2[1]

    returns = (price[i0[0]] * avg_monthly_returns[i0[0]] * i0[1] +
               price[i1[0]] * avg_monthly_returns[i1[0]] * i1[1] +
               price[i2[0]] * avg_monthly_returns[i2[0]] * i2[1])
    objective = alpha * risk - returns
    c0 = (price[i0[0]] * i0[1] +
          price[i1[0]] * i1[1] +
          price[i2[0]] * i2[1])

    print(price[i0[0]])
    print(2 * t_cost * price[i0[0]])
    print(2 * t_cost * price[i1[0]])
    print(2 * t_cost * price[i2[0]])

    lhs = (2 * t_cost  + (1 - t_cost)) * (price[i0[0]] * i0[1] + price[i1[0]] * i1[1] + price[i2[0]] * i2[1])

    print(f'lhs: {lhs}')
    print(f'budget: {budget}')
    print(f'test upper: {lhs <= budget}')
    print(f'test lower: {lhs >= budget - 0.003 * budget}')
    #print(2 * t_cost * price[i0[0]] * i0[1] + (price[i0[0]] * (1 - t_cost) * i0[1]))

    # t1 = price[i0[0]] * (1 - t_cost) * i0[1]
    # t1 = (price[i0[0]] * (1 - t_cost) * i0[1] +
    #                             price[i1[0]] * (1 - t_cost) * i1[1] +
    #                             price[i2[0]] * (1 - t_cost) * i2[1])

    # t0 = (t_cost * price[i0[0]] * i0[1] +
    #          t_cost * price[i1[0]] * i1[1] +
    #          t_cost * price[i2[0]] * i2[1])
    #
    # lhs = 2 * t0 + t1


    #c0 = t0 <= budget
    #c1 = t0 >= 0.997 * budget
    spending = (price[i0[0]] * max(0, i0[1]) +
                price[i1[0]] * max(0, i1[1]) +
                price[i2[0]] * max(0, i2[1]))

    variance = 0
    for i in [i0, i1, i2]:
        for j in [i0, i1, i2]:
                variance += price[i[0]] * i[1] * price[j[0]] * j[1] * covariance_matrix[i[0]][j[0]]

    # variance = 0.0
    # for s1, s2 in product(solution, solution):
    #     variance += (solution[s1] * self.price[s1]
    #                  * solution[s2] * self.price[s2]
    #                  * self.covariance_matrix[s1][s2])
    #
    # est_return = 0
    # for stock in solution:
    #     est_return += solution[stock]*self.price[stock]*self.avg_monthly_returns[stock]


    print(f'returns: {returns}')
    print(f'risk: {risk}')
    print(f'c0: {c0}')
    print(f'spending: {spending}')
    print(f'cost: {t_cost * (spending)}')
    print(f'variance: {variance}')
    #print(f't0: {t0}')
    #print(f't1: {t1}')
    #print(f'lhs: {lhs}')
    # lhs = 0
    # t0 = 0
    # t1 = (price[i0[0]] * (1 - t_cost) * i0[1] +
    #                            price[i1[0]] * (1 - t_cost) * i1[1] +
    #                            price[i2[0]] * (1 - t_cost) * i2[1] +
    #                            price[i3[0]] * (1 - t_cost) * i3[1])
    # # for b0 in [0,1]:
    # #     for b1 in [0,1]:
    # #         for b2 in [0,1]:
    # #             for b3 in [0,1]:
    # t0 = (t_cost * price[i0[0]] * i0[1] +
    #         t_cost * price[i1[0]] * i1[1] +
    #         t_cost * price[i2[0]] * i2[1] +
    #         t_cost * price[i3[0]] * i3[1])

    # lhs = 2 * t0 + t1
    # c0 = lhs <= budget
    # c1 = lhs >= budget - 0.003 * init_budget
    # print(b0, b1, b2, b3, lhs, c0, c1)
    # if c0 and c1: print(lhs)
#
# print(f'returns: {returns}')
# print(f'risk: {risk}')
# print(f'lhs: {lhs}')
# print(f'c0: {c0}')
# print(f'c1: {c1}')

157.418701171875
3.1483740234375
5.53750244140625
3.0487182617187503
lhs: 997.9245080566407
budget: 1000
test upper: True
test lower: True
returns: 29.496734659705517
risk: 505.26688176267714
c0: 988.0440673828125
spending: 988.0440673828125
cost: 9.880440673828126
variance: 3780.486736198098
